<a href="https://colab.research.google.com/github/mahir1995/nlp/blob/master/TextCNN_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

In [0]:
# TextCNN parameters
embedding_size = 2
seq_length = 3
num_classes = 2
filter_sizes = [2,2,2]
num_filters = 3

In [0]:
sentences = ["i love you", "he loves me", "she likes baseball", "i hate you", "sorry for that", "this is awful"]
labels = [1, 1, 1, 0, 0, 0] # 1 is good, 0 is bad
classes = ['bad', 'good']

In [6]:
word_list = list(set(' '.join(sentences).split()))
word_list

['hate',
 'for',
 'me',
 'loves',
 'sorry',
 'this',
 'that',
 'awful',
 'i',
 'baseball',
 'is',
 'likes',
 'love',
 'you',
 'he',
 'she']

In [7]:
word_dict = {w: i for i, w in enumerate(word_list)}
word_dict

{'awful': 7,
 'baseball': 9,
 'for': 1,
 'hate': 0,
 'he': 14,
 'i': 8,
 'is': 10,
 'likes': 11,
 'love': 12,
 'loves': 3,
 'me': 2,
 'she': 15,
 'sorry': 4,
 'that': 6,
 'this': 5,
 'you': 13}

In [8]:
vocab_size = len(word_dict)
vocab_size

16

In [9]:
inputs = []
for sen in sentences:
    inputs.append(np.asarray([word_dict[n] for n in sen.split()]))
inputs

[array([ 8, 12, 13]),
 array([14,  3,  2]),
 array([15, 11,  9]),
 array([ 8,  0, 13]),
 array([4, 1, 6]),
 array([ 5, 10,  7])]

In [11]:
targets = []
for out in labels:
    targets.append(out)
targets

[1, 1, 1, 0, 0, 0]

In [0]:
input_batch = Variable(torch.LongTensor(inputs))
target_batch = Variable(torch.LongTensor(targets))

In [0]:
class TextCNN(nn.Module):
    def __init__(self):
        super(TextCNN, self).__init__()
        
        self.num_filters_total = num_filters * len(filter_sizes)
        self.W = nn.Parameter(torch.empty(vocab_size, embedding_size).uniform_(-1, 1)).type(torch.FloatTensor)
        self.weight = nn.Parameter(torch.empty(self.num_filters_total, num_classes).uniform_(-1, 1)).type(torch.FloatTensor)
        self.bias = nn.Parameter(0.1 * torch.ones([num_classes])).type(torch.FloatTensor)
        
    def forward(self, x):
        embedded_chars = self.W[x]
        embedded_chars = embedded_chars.unsqueeze(1)
        
        pooled_outputs = []
        for filter_size in filter_sizes:
            conv = nn.Conv2d(1, num_filters, (filter_size, embedding_size), bias=True)(embedded_chars)
            h = F.relu(conv)
            mp = nn.MaxPool2d((seq_length - filter_size + 1, 1))
            pooled = mp(h).permute(0,3, 2, 1)
            pooled_outputs.append(pooled)
            
        h_pool = torch.cat(pooled_outputs, len(filter_sizes))
        h_pool_flat = torch.reshape(h_pool, [-1, self.num_filters_total])
        
        model = torch.mm(h_pool_flat, self.weight) + self.bias
        return model

In [0]:
model = TextCNN()

In [0]:
criterion = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(), lr = 0.001)

In [23]:
for epoch in range(5000):
    opt.zero_grad()
    output = model(input_batch)
    loss = criterion(output, target_batch)
    loss.backward()
    opt.step()
    
    if (epoch+1)%1000 == 0:
        print('Epoch:', epoch, 'Loss:', loss)

Epoch: 999 Loss: tensor(0.4168, grad_fn=<NllLossBackward>)
Epoch: 1999 Loss: tensor(0.1275, grad_fn=<NllLossBackward>)
Epoch: 2999 Loss: tensor(0.0489, grad_fn=<NllLossBackward>)
Epoch: 3999 Loss: tensor(0.0754, grad_fn=<NllLossBackward>)
Epoch: 4999 Loss: tensor(0.0190, grad_fn=<NllLossBackward>)


In [0]:
test = 'sorry i hate you'
test = [np.asarray([word_dict[n] for n in test.split()])]
test_batch = Variable(torch.LongTensor(test))

In [0]:
predict = model(test_batch)

In [46]:
prediction = predict.data.max(1, keepdim=True)[1]
classes[prediction]

'bad'